# Generating Causal Graphs with WhyNot

WhyNot comes equipped with tools to automatically generate *the causal graph* associated with experiments on the dynamical system simulators. Using ideas from recent work in [automatic differentiation](https://github.com/HIPS/autograd), WhyNot traces the simulator execution and the user-provided functions to select covariates, assign treatment, and compute the desired outcome to build up the corresponding causal graph. The graph is returned with the dataset as a Networkx graph object. 

This notebook demonstrates how to construct and manipulate the causal graph, and then combines the causal graph with the recent [DoWhy](https://github.com/microsoft/dowhy) package to perform causal inference using a graphical approach.


**Note**: This feature is still experimental, and there are likely a few rough edges. Only the HIV, Lotka-Volterra, and Opioid Epidemic simulators support causal graph generation at present. We are rapidly working on extending this feature to all of the dynamical system models.

In [1]:
%load_ext autoreload
%autoreload 2

import whynot as wn
import networkx as nx
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# To enable tracing of user-generated functions to construct the experiment,
# we have to import a thinly-wrapped version of numpy
import whynot.traceable_numpy as np

To showcase graph construction, we conduct a simple experiment on the Lotka-Volterra simulator to illustrate how to use causal graph construction. 
- The Lotka-Volterra simulator consists of two states, the rabbit population and the fox population. 
- The dynamics of the simulator are fully connected, so both the number of rabbits and foxes at time $t$ influence the number of rabbits at time $t+1$ and similarly for foxes. 
- We run the simulator for six time steps. 
- On the third time steps, we intervene to reduce the `fox_growth` parameter, i.e. the factor describing how rabbits are needed to sustain a fox. 
- To generate confounding, treatment is more likely when the fox poulation at time step 3 is low. 
- The outcome is the total fox population at time 6.
- The observed covariates is the fox and rabbit population at the time of intervention.

In [3]:
# User-defined helper functions for the experiment

def sample_initial_states(rng):
    """Initial state distribution"""
    rabbits = rng.randint(10, 100)
    # Ensure the number of rabbits is greater than number of foxes.
    foxes = rng.uniform(0.1, 0.8) * rabbits
    return wn.lotka_volterra.State(rabbits=rabbits, foxes=foxes)

def soft_threshold(x, threshold, r=20):
    """A continuous relaxation of the threshold function. If x > tau, return ~1, if x < tau, returns ~0."""
    return 1. / (np.exp(r * (threshold  - x)) + 1)


def confounded_propensity_scores(untreated_run, intervention):
    """Return confounded treatment assignment probability.
    Treatment is more likely for runs with low initial fox population.
    """
    return 0.3 + 0.4 * (1. - soft_threshold(untreated_run[intervention.time].foxes, threshold=7))

def covariate_observer(run, intervention):
    """Return the full state at the time step before and at the time of intervention."""
    prev_state = run[max(intervention.time - 1, 0)].values()
    curr_state = run[intervention.time].values()
    return np.concatenate([prev_state, curr_state])

def outcome_extractor(run, config):
    """Final fox population."""
    return run[config.end_time].foxes

In [4]:
# Construct the experiment
exp = wn.DynamicsExperiment(
    name="lotka_volterra_confounding",
    description=("Determine effect of reducing rabbits needed to sustain a fox. Treament confounded by initial fox population."),
    simulator=wn.lotka_volterra,
    simulator_config=wn.lotka_volterra.Config(fox_growth=0.75, delta_t=1, end_time=6),
    intervention=wn.lotka_volterra.Intervention(time=3, fox_growth=0.4),
    state_sampler=sample_initial_states,
    propensity_scorer=confounded_propensity_scores,
    outcome_extractor=outcome_extractor,
    covariate_builder=covariate_observer)

To generate the causal graph associated with the experiment, pass `causal_graph=True` into the `experiment.run` function.

In [5]:
dset = exp.run(num_samples=100, causal_graph=True)

In [6]:
# The causal graph is a networkx object
graph = dset.causal_graph

In [7]:
# The nodes in the causal graphs are the state variables for the unrolled dynamics,
# the configuration parameters, the treatment, and the outcome
graph.nodes

NodeView(('rabbits_0', 'foxes_0', 'rabbit_growth_0', 'rabbit_death_0', 'fox_death_0', 'fox_growth_0', 'rabbits_1', 'foxes_1', 'rabbit_growth_1', 'rabbit_death_1', 'fox_death_1', 'fox_growth_1', 'rabbits_2', 'foxes_2', 'rabbit_growth_2', 'rabbit_death_2', 'fox_death_2', 'fox_growth_2', 'rabbits_3', 'foxes_3', 'rabbit_growth_3', 'rabbit_death_3', 'fox_death_3', 'fox_growth_3', 'rabbits_4', 'foxes_4', 'rabbit_growth_4', 'rabbit_death_4', 'fox_death_4', 'fox_growth_4', 'rabbits_5', 'foxes_5', 'rabbit_growth_5', 'rabbit_death_5', 'fox_death_5', 'fox_growth_5', 'rabbits_6', 'foxes_6', 'rabbit_growth_6', 'rabbit_death_6', 'fox_death_6', 'fox_growth_6', 'Treatment', 'Outcome'))

In [8]:
# Treatment only directly depends on the fox population at time 3
graph.in_edges("Treatment")

InEdgeDataView([('foxes_3', 'Treatment')])

In [9]:
# Treatment affects fox_growth at all times steps >=3 
graph.out_edges("Treatment")

OutEdgeDataView([('Treatment', 'fox_growth_3'), ('Treatment', 'fox_growth_4'), ('Treatment', 'fox_growth_5'), ('Treatment', 'fox_growth_6')])

In [10]:
# The outcome only depends on the final fox population
graph.in_edges("Outcome")

InEdgeDataView([('foxes_6', 'Outcome')])

In [11]:
# The observed covariates are foxes and rabbits at the step before intervention, i.e. step 2
for node, data in graph.nodes.items():
    if data["observed"] == "yes":
        print(node)

rabbits_2
foxes_2
rabbits_3
foxes_3
Treatment
Outcome


With access to the causal graph, we can use causal inference methods based on graphical analysis. We make use of the [DoWhy package](https://github.com/microsoft/dowhy) to demonstrate how combine data from WhyNot with graphical methods.

In [12]:
# Convert the dataset into a pandas datafrom
data = np.concatenate([dset.covariates, dset.treatments.reshape(-1, 1), dset.outcomes.reshape(-1, 1)], axis=1)
df = pd.DataFrame(data, columns=graph.graph["covariate_names"] + ["Treatment", "Outcome"])
df.head()

,rabbits_2,foxes_2,rabbits_3,foxes_3,Treatment,Outcome
0,5.251042,10.091993,7.699480,3.550094,1.0,2.766330
1,0.992871,7.090575,1.846604,1.746722,1.0,0.068187
2,3.218797,6.540883,5.994306,2.015959,0.0,13.268573
3,0.266265,7.378253,0.492295,1.690392,0.0,0.035348
4,0.586202,7.098729,1.095395,1.679466,0.0,0.075772


In [ ]:
# Install DoWhy
!pip install dowhy

In [14]:
from dowhy.causal_model import CausalModel

In [17]:
# DoWhy expects a direct edge from Treatment->Outcome
graph.add_edges_from([("Treatment", "Outcome")])
del graph.graph["covariate_names"]
nx.write_gml(graph, "lotka_volterra_confounding.gml")

In [18]:
model = CausalModel(
    data=df,
    treatment="Treatment",
    outcome="Outcome",
    graph="lotka_volterra_confounding.gml")

INFO:dowhy.causal_model:Model to find the causal effect of treatment ['Treatment'] on outcome ['Outcome']


In [19]:
# Identify causal effect and return target estimands. There are unobserved common causes,
# but there exists a set of variables that blocks all backdoor paths. DoWhy doesn't seem to account for this?
identified_estimand = model.identify_effect()
print(identified_estimand)

INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['fox_death_1', 'rabbit_growth_0', 'rabbit_death_1', 'rabbits_1', 'foxes_1', 'foxes_0', 'rabbits_2', 'fox_growth_1', 'rabbit_growth_1', 'foxes_3', 'foxes_2', 'fox_death_2', 'fox_growth_2', 'fox_death_0', 'rabbits_0', 'rabbit_death_2', 'fox_growth_0', 'rabbit_death_0']


WARN: Do you want to continue by ignoring these unobserved confounders? [y/n] y


INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]


{'estimand': Derivative(Expectation(Outcome|rabbits_2,foxes_3,foxes_2), Treatment), 'assumptions': {'Unconfoundedness': 'If U→Treatment and U→Outcome then P(Outcome|Treatment,rabbits_2,foxes_3,foxes_2,U) = P(Outcome|Treatment,rabbits_2,foxes_3,foxes_2)'}}
Estimand type: ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
    d                                                     
──────────(Expectation(Outcome|rabbits_2,foxes_3,foxes_2))
dTreatment                                                
Estimand assumption 1, Unconfoundedness: If U→Treatment and U→Outcome then P(Outcome|Treatment,rabbits_2,foxes_3,foxes_2,U) = P(Outcome|Treatment,rabbits_2,foxes_3,foxes_2)
### Estimand : 2
Estimand name: iv
No such variable found!



In [20]:
# Estimate the target estimand using a statistical method.
estimate = model.estimate_effect(identified_estimand,
                                 method_name="backdoor.propensity_score_matching")
print(estimate)

INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: Outcome~Treatment+rabbits_2+foxes_3+foxes_2


*** Causal Estimate ***

## Target estimand
Estimand type: ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
    d                                                     
──────────(Expectation(Outcome|rabbits_2,foxes_3,foxes_2))
dTreatment                                                
Estimand assumption 1, Unconfoundedness: If U→Treatment and U→Outcome then P(Outcome|Treatment,rabbits_2,foxes_3,foxes_2,U) = P(Outcome|Treatment,rabbits_2,foxes_3,foxes_2)
### Estimand : 2
Estimand name: iv
No such variable found!

## Realized estimand
b: Outcome~Treatment+rabbits_2+foxes_3+foxes_2
## Estimate
Value: -7.655575689477137



/Users/JohnMiller/research/dowhy/dowhy/causal_estimators/propensity_score_matching_estimator.py:51: FutureWarning: `item` has been deprecated and will be removed in a future version
  control_outcome = control.iloc[indices[i]][self._outcome_name].item()
